In [221]:
def cal_percentile(x: list, k: int):
    import numpy as np
    n = len(x)

    if n > 100:
        p = np.percentile(x, k)
    else:
        x = sorted(x)
        i = (k / 100) * (n - 1)
        j = int(i)

        if i == j:
            p = x[j]
        else: 
            p = x[j] + (i - j) * (x[j + 1] - x[j])
    return p

In [222]:
x = [1,7,2,6]
k = 99.5
p = cal_percentile(x, k)
p

6.984999999999999

In [9]:
import json

def write_dict_to_json(file_path, data):
    with open(file_path, 'wb') as file:
        file.write(json.dumps(data)) 

In [7]:
import json

def read_json(file_path):
    with open(file_path, 'rb') as file:  # Mở file dưới dạng nhị phân
        return json.loads(file.read())  # Đọc và chuyển đổi từ bytes thành Python object

# Đọc file JSON
file_path = 'output.json'
data = read_json(file_path)
print(data)


{'2': [1, 7, 2, 6], '1235465': [1, 7, 2, 6, 1, 7, 2, 6]}


In [15]:
file_path = './output.json'
with open(file_path, 'rb') as file:
    data = json.loads(file.read()) 
    data = {int(k): v for k, v in data.items()}

In [17]:
{int(k): v for k, v in data.items()}

{123546: [1.0, 7.0, 2.0, 6.0]}

In [142]:
import pickle
import hashlib
import bisect
import os

class PickleShardManager:
    def __init__(self, num_shards=3, folder="data_shards"):
        self.num_shards = num_shards
        self.folder = folder
        self.metadata_file = f"{folder}/metadata.pkl"  # File lưu metadata
        self.shard_keys = []
        self.shards = {}
        self.metadata = {}

        os.makedirs(folder, exist_ok=True)  # Tạo folder nếu chưa có

        # Tạo các file shard với hash key
        for i in range(num_shards):
            shard_key = self._hash(f"shard-{i}")  # Hash để phân bố shard
            self.shard_keys.append(shard_key)
            self.shards[shard_key] = f"{folder}/shard_{i}.pkl"

        self.shard_keys.sort()  # Sắp xếp để dùng `bisect`

        # Load metadata nếu có
        self._load_metadata()

    def _hash(self, key):
        """Tạo hash integer từ key"""
        return int(hashlib.md5(key.encode()).hexdigest(), 16)

    def _get_shard_file(self, key):
        """Tìm file pickle phù hợp cho key"""
        key_hash = self._hash(key)
        idx = bisect.bisect(self.shard_keys, key_hash) % self.num_shards
        return self.shards[self.shard_keys[idx]]

    def _save_metadata(self):
        """Lưu metadata vào file"""
        with open(self.metadata_file, "wb") as f:
            pickle.dump(self.metadata, f)

    def _load_metadata(self):
        """Tải metadata từ file nếu tồn tại"""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, "rb") as f:
                self.metadata = pickle.load(f)

    def save(self, key, data):
        """Lưu dict vào file pickle phù hợp và cập nhật metadata"""
        file_path = self._get_shard_file(key)

        # Cập nhật metadata
        self.metadata[key] = file_path
        self._save_metadata()

        # Lưu vào file pickle
        with open(file_path, "ab") as f:
            pickle.dump({key: data}, f)

    def exists(self, key):
        """Kiểm tra nhanh xem key có tồn tại không"""
        return key in self.metadata  # Kiểm tra ngay trong metadata

    def update(self, key, new_data):
        """Cập nhật dữ liệu của một key"""
        if key not in self.metadata:
            raise KeyError(f"Key '{key}' không tồn tại!")

        file_path = self.metadata[key]

        # Đọc toàn bộ dữ liệu từ file shard
        data = self.load_shard(file_path)
        data[key] = new_data  # Cập nhật giá trị

        # Ghi đè lại file với dữ liệu mới
        with open(file_path, "wb") as f:
            pickle.dump(data, f)

    def load_shard(self, file_path):
        """Đọc toàn bộ dữ liệu từ một shard"""
        data = {}
        if os.path.exists(file_path):
            with open(file_path, "rb") as f:
                while True:
                    try:
                        record = pickle.load(f)
                        data.update(record)
                    except EOFError:
                        break
        return data

    def load_all(self):
        """Đọc toàn bộ dữ liệu từ tất cả các shards"""
        all_data = {}
        for file_path in self.shards.values():
            all_data.update(self.load_shard(file_path))
        return all_data

# ------------------ Test ------------------
manager = PickleShardManager(num_shards=3)

# Lưu dữ liệu
manager.save("1", {"name": "Alice", "age": 25})
manager.save("2", {"product": "Laptop", "price": 1200})

# Kiểm tra key có tồn tại không (siêu nhanh)
print(manager.exists("1"))  # Output: True
print(manager.exists("2"))  # Output: True
print(manager.exists("product999"))  # Output: False

# Cập nhật dữ liệu
manager.update("user123", {"name": "Alice", "age": 26})

# Load toàn bộ dữ liệu
print(manager.load_all())


True
True
False
{'order45612313123': {'product': 'Laptop', 'price': 1200}, 'pr4234242oduct789': {'category': 'Electronics'}, 'order456': {'product': 'Laptop', 'price': 1200}, '1': {'name': 'Alice', 'age': 25}, '2': {'product': 'Laptop', 'price': 1200}, 'user123': {'name': 'Alice', 'age': 26}, 'product789': {'category': 'Electronics'}}


In [115]:
# Test
pool = PoolManager(pool_id=1, pool_values=[10.5, 20.3, 30.7])

In [116]:
pool.save_pickle(file_path="./data.pkl")

In [117]:
pool.read_pickle(file_path="./data.pkl")

{'pool_id': 1, 'pool_values': [10.5, 20.3, 30.7]}
